# Análise de Séries Temporais com Redes Neurais Recorrentes

Neste notebook, vamos construir uma Rede Neural Recorrente (RNN) do tipo LSTM (Long Short-Term Memory) para analisar três conjuntos de dados de séries temporais:

- Alcohol_Sales.csv
- Miles_Traveled.csv
- BeerWineLiquor.csv

Vamos começar importando as bibliotecas necessárias e carregando os conjuntos de dados.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, GRU
from keras.callbacks import ModelCheckpoint

# Carregar os conjuntos de dados
alcohol_sales = pd.read_csv('Alcohol_Sales.csv')
miles_traveled = pd.read_csv('Miles_Traveled.csv')
beer_wine_liquor = pd.read_csv('BeerWineLiquor.csv')

## Análise Exploratória de Dados (EDA)

Vamos realizar uma análise exploratória básica para entender melhor nossos conjuntos de dados.

In [ ]:
# Verificar as primeiras linhas de cada conjunto de dados
print('Alcohol Sales:')
print(alcohol_sales.head())
print('\nMiles Traveled:')
print(miles_traveled.head())
print('\nBeer, Wine, Liquor:')
print(beer_wine_liquor.head())

# Verificar a dimensão de cada conjunto de dados
print('\nShape of the datasets:')
print('Alcohol Sales:', alcohol_sales.shape)
print('Miles Traveled:', miles_traveled.shape)
print('Beer, Wine, Liquor:', beer_wine_liquor.shape)

## Pré-processamento dos Dados

Antes de alimentar os dados para a rede neural, precisamos realizar algumas etapas de pré-processamento. Isso inclui a divisão dos dados em conjuntos de treinamento e teste, a normalização dos dados e a transformação dos dados em um formato adequado para a rede neural.

In [ ]:
# Função para pré-processar os dados
def preprocess_data(data, test_size=0.2):
    # Dividir os dados em conjuntos de treinamento e teste
    train_data, test_data = train_test_split(data, test_size=test_size, shuffle=False)

    # Normalizar os dados
    scaler = MinMaxScaler()
    train_data = scaler.fit_transform(train_data)
    test_data = scaler.transform(test_data)

    # Transformar os dados para o formato adequado para a rede neural
    # O formato é (número de amostras, passos de tempo, número de características)
    train_data = np.reshape(train_data, (train_data.shape[0], 1, train_data.shape[1]))
    test_data = np.reshape(test_data, (test_data.shape[0], 1, test_data.shape[1]))

    return train_data, test_data, scaler

# Pré-processar os conjuntos de dados
alcohol_sales_train, alcohol_sales_test, alcohol_sales_scaler = preprocess_data(alcohol_sales)
miles_traveled_train, miles_traveled_test, miles_traveled_scaler = preprocess_data(miles_traveled)
beer_wine_liquor_train, beer_wine_liquor_test, beer_wine_liquor_scaler = preprocess_data(beer_wine_liquor)

## Construção do Modelo LSTM

Agora que nossos dados estão prontos, podemos construir o modelo LSTM. Vamos criar uma função para isso, para que possamos reutilizá-la para cada conjunto de dados.

In [ ]:
# Função para construir o modelo LSTM
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Construir o modelo LSTM para cada conjunto de dados
alcohol_sales_model = build_lstm_model((1, alcohol_sales_train.shape[2]))
miles_traveled_model = build_lstm_model((1, miles_traveled_train.shape[2]))
beer_wine_liquor_model = build_lstm_model((1, beer_wine_liquor_train.shape[2]))

## Treinamento do Modelo

Vamos treinar nossos modelos LSTM. Para cada modelo, vamos usar um callback ModelCheckpoint para salvar o modelo com a menor perda de validação.

In [ ]:
# Função para treinar o modelo
def train_model(model, train_data, test_data, epochs=10, batch_size=32):
    # Callback para salvar o modelo com a menor perda de validação
    checkpoint = ModelCheckpoint('model.h5', save_best_only=True, monitor='val_loss', mode='min')

    # Treinar o modelo
    history = model.fit(train_data, epochs=epochs, batch_size=batch_size, validation_data=test_data, callbacks=[checkpoint], verbose=0)

    return model, history

# Treinar os modelos LSTM
alcohol_sales_model, alcohol_sales_history = train_model(alcohol_sales_model, alcohol_sales_train, alcohol_sales_test)
miles_traveled_model, miles_traveled_history = train_model(miles_traveled_model, miles_traveled_train, miles_traveled_test)
beer_wine_liquor_model, beer_wine_liquor_history = train_model(beer_wine_liquor_model, beer_wine_liquor_train, beer_wine_liquor_test)

## Avaliação do Modelo

Após o treinamento, vamos avaliar o desempenho de nossos modelos. Vamos plotar a perda de treinamento e validação ao longo das épocas e também calcular o erro quadrático médio (MSE) no conjunto de teste.

In [ ]:
# Função para plotar a perda de treinamento e validação
def plot_loss(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')
    plt.show()

# Função para calcular o MSE
def calculate_mse(model, test_data):
    predictions = model.predict(test_data)
    mse = np.mean((predictions - test_data)**2)
    return mse

# Avaliar os modelos LSTM
print('Alcohol Sales Model:')
plot_loss(alcohol_sales_history)
print('MSE:', calculate_mse(alcohol_sales_model, alcohol_sales_test))

print('\nMiles Traveled Model:')
plot_loss(miles_traveled_history)
print('MSE:', calculate_mse(miles_traveled_model, miles_traveled_test))

print('\nBeer, Wine, Liquor Model:')
plot_loss(beer_wine_liquor_history)
print('MSE:', calculate_mse(beer_wine_liquor_model, beer_wine_liquor_test))

## Comparação com Modelos Simples de RNN e GRU

Vamos comparar o desempenho de nossos modelos LSTM com modelos de RNN simples e GRU. Para isso, vamos construir e treinar esses modelos para cada conjunto de dados.

In [ ]:
# Função para construir o modelo SimpleRNN
def build_simplernn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Função para construir o modelo GRU
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Construir os modelos SimpleRNN e GRU para cada conjunto de dados
alcohol_sales_simplernn_model = build_simplernn_model((1, alcohol_sales_train.shape[2]))
alcohol_sales_gru_model = build_gru_model((1, alcohol_sales_train.shape[2]))

miles_traveled_simplernn_model = build_simplernn_model((1, miles_traveled_train.shape[2]))
miles_traveled_gru_model = build_gru_model((1, miles_traveled_train.shape[2]))

beer_wine_liquor_simplernn_model = build_simplernn_model((1, beer_wine_liquor_train.shape[2]))
beer_wine_liquor_gru_model = build_gru_model((1, beer_wine_liquor_train.shape[2]))

## Comparação com Modelos ARIMA e SARIMA

Finalmente, vamos comparar o desempenho de nossos modelos de redes neurais com modelos clássicos de séries temporais, como ARIMA e SARIMA. Para isso, vamos construir e treinar esses modelos para cada conjunto de dados.

In [ ]:
# Função para construir e treinar o modelo ARIMA
def build_arima_model(data, order):
    model = ARIMA(data, order=order)
    model_fit = model.fit(disp=0)
    return model_fit

# Função para construir e treinar o modelo SARIMA
def build_sarima_model(data, order, seasonal_order):
    model = SARIMAX(data, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit(disp=0)
    return model_fit

# Construir e treinar os modelos ARIMA e SARIMA para cada conjunto de dados
alcohol_sales_arima_model = build_arima_model(alcohol_sales_train, (5,1,0))
alcohol_sales_sarima_model = build_sarima_model(alcohol_sales_train, (5,1,0), (1,1,1,12))

miles_traveled_arima_model = build_arima_model(miles_traveled_train, (5,1,0))
miles_traveled_sarima_model = build_sarima_model(miles_traveled_train, (5,1,0), (1,1,1,12))

beer_wine_liquor_arima_model = build_arima_model(beer_wine_liquor_train, (5,1,0))
beer_wine_liquor_sarima_model = build_sarima_model(beer_wine_liquor_train, (5,1,0), (1,1,1,12))